In [7]:
from tokenizer import Tokenizer
import torch
import torch.nn as nn
import torch.nn as nn
from torch.nn import functional as f
torch.manual_seed(42)

In [16]:
obj = Tokenizer()
obj.character_tokenizer()
data=torch.tensor(obj.encoder(s=obj.data),dtype=torch.int64)

In [17]:
n=int(0.9*len(data))
traindata = data[:n]
intrimdata = data[n:]

In [18]:
block_size = 8  # context length
batch_size = 4   # number of sequences in a batch
def getbatch_train_data():
    data = torch.tensor(traindata) 
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

def getbatch_test_data():
    data = torch.tensor(intrimdata) 
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

In [78]:
class BigramLanguagemodel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(embedding_dim=vocab_size,num_embeddings=vocab_size)

    def forward(self,idx,targets=None):
        logits = self.token_embedding(idx)
        if  targets != None:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = f.cross_entropy(logits,target=targets)
        else:
            loss=None
        return logits,loss
    
    def generate(self, idx, max_newtokens):
        for i in range(max_newtokens):
            logits, _= self(idx)
            logits = logits[:,-1,:]
            probs = f.softmax(logits,dim=1)
            id_next = torch.multinomial(probs,num_samples=1)
            idx = torch.cat((idx, id_next), dim=1)
        return idx
    
otg=BigramLanguagemodel(vocab_size=obj.vocab_size)


In [ ]:
optimizer = torch.optim.AdamW(otg.parameters(),lr=1e-3)

In [ ]:
batch_size = 32
for i in 